In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import pandas_datareader.data as web
import time
import tensorflow
import matplotlib.pyplot as plt
import datetime as dt
import numpy as np
import logging
import math
import os
from sklearn.preprocessing import MinMaxScaler, scale
from keras.utils import np_utils
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
from keras import optimizers
from sklearn.metrics import mean_squared_error
from keras.models import load_model
from tqdm import tqdm_notebook

Using TensorFlow backend.


In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 500)

# 데이터 불러오기 / 데이터 정리

In [3]:
IT_finance = pd.read_excel('./data/dongboo/IT_TOTAL_재무사항.xlsx')
Medicine_finance = pd.read_excel('./data/dongboo/제약_TOTAL_재무사항.xlsx')

In [4]:
# krx에서 가져온 자료
IT_stock = pd.read_excel('./data/dongboo/IT_주가_total_krx.xlsx')
Medicine_stock = pd.read_excel('./data/dongboo/제약_주가_total_krx.xlsx')

# IT_stock = pd.read_excel('./data/dongboo/IT_TOTAL_주가.xlsx')
# Medicine_stock = pd.read_excel('./data/dongboo/제약_TOTAL_주가.xlsx')

In [5]:
IT_stock

,회사명,거래소코드,회계년도,최고가,최저가,종가,거래량,거래대금
0,LG이노텍(주),11070,2018/12,103000,84000,93500,3739997,345607299000
1,LG이노텍(주),11070,2018/11,127000,94000,110500,7990160,849334038995
2,LG이노텍(주),11070,2018/10,133000,118000,125500,4229243,533333395000
3,LG이노텍(주),11070,2018/09,155500,130000,142750,5569000,764547735607
4,LG이노텍(주),11070,2018/08,164000,137000,150500,4849283,711882140750
5,LG이노텍(주),11070,2018/07,163000,139500,151250,4096286,639917556000
6,LG이노텍(주),11070,2018/06,155500,139000,147250,4568478,671827839768
7,LG이노텍(주),11070,2018/05,144500,123500,134000,5437043,725054123493
8,LG이노텍(주),11070,2018/04,136500,116500,126500,6099695,763215432058
9,LG이노텍(주),11070,2018/03,137500,120000,128750,5184190,668888538140


In [6]:
IT = pd.merge(IT_finance,IT_stock,how = 'left', on = ['회사명','거래소코드','회계년도'])
Medicine = pd.merge(Medicine_finance,Medicine_stock,how = 'left', on = ['회사명','거래소코드','회계년도'])

In [7]:
del IT['주식매입선택권.1']

# 회사별로 분리

In [8]:
lg = IT[IT['회사명'] == 'LG이노텍(주)']
sdi = IT[IT['회사명'] == '삼성SDI(주)']
ssem = IT[IT['회사명'] == '삼성전기(주)']
hynix = IT[IT['회사명'] == '에스케이하이닉스(주)']

In [9]:
lg

,회사명,거래소코드,회계년도,자산(*),자본(*),* 수권주식수,*주당 액면가액,* 발행한 주식총수(*),보통주,우선주,지배기업주주지분(*),자본금(*),보통주자본금,우선주자본금,신종자본증권,자본잉여금(*),주식발행초과금,배당건설이자,주식매입선택권,감자차익,보험차익,합병차익,자산수증이익,채무면제이익,자기주식처분이익,전환권대가,신주인수권대가,교환권대가,전환권재매입이익,기타자본잉여금,재평가적립금(2001년 이전),기타자본(*),자기주식,자기주식처분손실,주식할인발행차금,감자차손,파생상품평가이익,파생상품평가손실,전환권재매입손실,기타,기타포괄손익누계액(*),금융자산평가손익,매도가능금융자산평가손익,관계기업 등 기타포괄손익,환산 관련 외환차이 적립금(환율변동차이),해외사업장순투자위험회피 적립금,확정급여제도의 보험수리적 손익 적립금,확정급여제도의 재측정요소,현금흐름위험회피 적립금,주식기준보상 적립금,재평가잉여금,기타.1,이익잉여금(결손금)(*),* (지배)당기순손익,이익준비금,재무구조개선적립금,기업합리화적립금,기타법정적립금,재평가손익,보험수리적손익,기타이익잉여금,미처분이익잉여금(결손금),*전기이월미처분이익잉여금(결손금),비지배주주지분,* (비지배)당기순손익,*비지배주주지분초과손실액,부채(*),* (정상)영업손익(보고서기재),* (정상)영업손익(계산수치),기타이익(*),이자수익,배당금수익,외환거래이익(*),외환차익,외환환산이익,기타외화거래이익,보험차익.1,퇴직연금운용이익,전기오류수정이익,기타.2,기타손실(*),이자비용,외환거래손실(*),외환차손,외화환산손실,기타외환거래손실,법인세추납액,퇴직연금운용손실,전기오류수정손실,기타.3,법인세비용차감전순이익(손실),법인세비용,종속회사 매수일전 순손익,처분된 종속회사 순손익,계속영업이익(손실),중단영업이익(손실),*중단영업손익 법인세효과,당기순이익(손실),기타포괄손익(*),금융자산평가손익.1,매도가능금융자산평가손익.1,관계기업 등 기타포괄손익.1,환산 관련 외환차이(환율변동차이),해외사업장순투자위험회피,현금흐름위험회피 적립금.1,확정급여제도의 보험수리적손익,확정급여제도의 재측정요소.1,주식기준보상 적립금.1,재평가손익.1,기타.4,기타포괄손익관련 법인세,총포괄손익,*(당기순손익 귀속)지배기업주주지분,*계속영업손익,*중단영업손익,*(당기순손익 귀속)비지배주주지분,*계속영업손익.1,*중단영업손익.1,*(총포괄손익 귀속)지배기업주주지분,*(총포괄손익 귀속)비지배주주지분,*주당계속영업이익,*주당순손익,*희석주당계속영업이익,*희석주당순손익,*(지배기업주주지분)주당계속영업이익,*(지배기업주주지분)주당순손익,*(지배기업주주지분)희석주당계속영업이익,*(지배기업주주지분)희석주당순손익,* 성격별 비용계정(*),제품과 재공품의 변동,기업이 수행한 용역으로서 자본화되어 있는 부분,원재료와 저장품(소모품)의 사용액,상품의 판매,기타원가,종업원 급여비용,감가상각비와 기타상각비 및 손상차손,세금과공과,대손상각비,물류비(운송보관),광고 및 판매촉진비,임차료 및 리스료,경상연구개발비,기타비용,기말자본금(*),기초자본금,유상증자(감자),무상증자(감자),주식매수선택권,전환사채,신주인수권부사채,주식배당,합병분할(영업양수도 등)로 인한 변동,회계정책변경누적효과,오류수정 등,연결범위의 변동,외화환산차이,기타.5,기말신종자본증권(*),기초신종자본증권,신종자본증권의 발행,기타.6,기말자본잉여금(*),기초자본잉여금,유상증자(감자).1,무상증자(감자).1,주식발행초과금.1,주식매수선택권.1,감자차손익,보험차익.2,자기주식처분손익,전환권대가(전환사채),신주인수권대가(신주인수권부사채),교환권대가(교환사채),전환권재매입손익,결손금처리,지분법자본잉여금,합병분할(영업양수도 등)로 인한 변동.1,회계정책변경누적효과.1,오류수정 등.1,연결범위의 변동.1,외화환산차이.1,기타.7,기말기타자본(*),기초기타자본,자기주식.1,자기주식처분손실.1,주식할인발행차금.1,주식매입선택권.2,파생상품평가이익.1,파생상품평가손실.1,출자전환채무,신주청약증거금,감자차손익.1,전환권재매입손익.1,지분법자본조정,합병분할(영업양수도 등)로 인한 변동.2,회계정책변경누적효과.2,오류수정 등.2,연결범위의 변동.2,외화환산차이.2,기타.8,기말기타포괄손익누계액(*),기초기타포괄손익누계액,금융자산평가손익.2,매도가능금융자산 평가손익,관계기업 등 기타포괄손익.2,환산 관련 외환차이 적립금(환율변동차이).1,해외사업장순투자위험회피 적립금.1,확정급여제도의 보험수리적 손익 적립금.1,확정급여제도의 재측정요소.2,현금흐름위험회피 적립금.2,주식기준보상 적립금.2,재평가잉여금.1,합병분할(영업양수도 등)로 인한 변동.3,회계정책변경누적효과.3,오류수정 등.3,연결범위의 변동.3,외화환산차이.3,기타.9,기말이익잉여금(결손금)(*),처분후 이익잉여금(결손금)(*),수정후 이익잉여금(결손금)(*),기초이익잉여금(결손금),회계정책변경누적효과.4,전기오류수정손익,기타.10,(연차배당),(신종자본증권 분배금),주식할인발행차금의 상각,자기주식처분손실의 상각,결손금처리.1,기타이익잉여금처분액,(중간배당),확정급여제도의 재측정요소.3,기타포괄손익,당기순이익(총포괄손익),보험수리적손익.1,재평가손익.2,지분법이익잉여금,합병분할(영업양수도 등)로 인한 변동.4,연결범위의 변동.4,외화환산차이.4,기타.11,기말비지배주주지분(*),처분후 비지배주주지분(*),수정후 비지배주주지분(*),기초 비지배주주지분,회계정책변경누적효과.5,전기오류수정손익.1,기타.12,(종속회사의 배당금),기타.13,(중간배당).1,당기순이익(총포괄손익).1,재평가잉여금.2,금융자산평가손익.3,매도가능금융자산 평가손익.1,관계기업 등 지분법평가손익,환산 관련 외환차이 적립금(환율변동차이).2,해외사업장순투자위험회피 적립금.2,확정급여제도의 보험수리적 손익 적립금.2,확정급여제도의 재측정요소.4,현금흐름위험회피 적립금.3,주식기준보상 적립금.3,기타포괄손익누계액,합병분할로 인한 변동,연결범위의 변동.5,외화환산차이.5,기타.14,합계,영업활동으로 인한 현금흐름(간접법)(*),*(직접법)총현금유입(*),재화의 판매와 용역제공에 따른 현금유입,"수입보험료, 보험금, 연금 및 기타 급부금관련 현금유입",이자수취액,배당금수취액,법인세환급,기타현금유입액,*(직접법)총현금유출(*),종업원과 관련하여 직·간접으로 발생하는 현금유출,"수입보험료, 보험금, 연금 및 기타 급부금관련 현금유출",이자비용유출액,배당금지급액,법인세납부,단기매매목적으로 보유하는 계약에서 발생하는 현금유출,기타현금유출액,당기순손익,법인세비용차감전순손익,현금 유출이 없는 비용 등 가산(*),전기오류수정손익.2,비지배주주지분순손익,보험수리적손실,기타.15,(현금 유입이 없는 수익 등 차감)(*),전기오류수정손익.3,비지배주주지분순손익.1,보험수리적이익,기타.16,영업활동으로 인한 자산부채 변동(운전자본 변동)(*),자산의 감소(증가)(*),부채의 증가(감소)(*),정부보조금 등의 변동,기타 운전자본의 변동,*영업에서 창출된 현금흐름,이자수입,이자지급(-),배당금수입,배당금지급(-),법인세환입,법인세납부(-),중단영업관련 현금흐름,투자활동으로 인한 현금흐름(*),투자활동으로 인한 현금유입액(*),정부보조금 등의 변동.1,종속기업 및 기타사업의 지배력 관련한 현금흐름,합병분할(양수도등)으로 인한 현금유입액,기타투자활동으로 인한 현금유입액,(투자활동으로 인한 현금유출액)(*),정부보조금 등의 변동.2,종속기업 및 기타사업의 지배력 관련한 현금흐름.1,합병분할(양수도등)으로 인한 현금유출액,기타투자활동으로 인한 현금유출액,이자수입.1,이자지급(-).1,배당금수입.1,배당금지급(-).1,법인세환입.1,법인세납부(-).1,중단영업관련 현금흐름.1,재무활동으로 인한 현금흐름(*),재무활동으로 인한 현금유입액(*),기타부채의 증가,유상증자,자본금의 증가,자본잉여금의 증가,종속회사의 증자 등,이익잉여금의 증가,자기주식의 처분,주식매입선택권의 행사,연결자본거래로인한현금유입액,종속기업 소유지분 변동에 따른 현금흐름,비지배주주지분의 증가,정부보조금 등의 변동.3,환율변동으로인한 차이조정,합병분할(양수도등)으로 인한 현금유입액.1,기타재무활동으로 인한 현금유입액,(재무활동으로 인한 현금유출액)(*),유상감자,자본금의

In [10]:
greencross = Medicine[Medicine['회사명'] == '(주)녹십자']
daewoong = Medicine[Medicine['회사명'] == '(주)대웅제약']
yoohan = Medicine[Medicine['회사명'] == '(주)유한양행']
hanmi = Medicine[Medicine['회사명'] == '한미약품(주)']

In [11]:
def classify(stock):
    
    # 회계년도 순으로 정렬 및 인덱스 초기화
    stock.sort_values('회계년도', ascending= True, inplace = True)
    
    stock.reset_index(drop = True, inplace = True)
    
    
    # 새로운 변수 추가
    
    stock['3개월후종가'] = stock['종가'][1:].reset_index(drop=True)
    
    stock['분기수익률'] = (stock['3개월후종가'] - stock['종가']) / stock['종가']
    
    stock['수익률상승하락'] = stock['분기수익률']
    
    for i in range(len(stock)):
        if stock['분기수익률'][i] < 0.005:
            stock['수익률상승하락'][i] = 0
        else:
            stock['수익률상승하락'][i] = 1
        
       
    return stock

In [12]:
classify(lg)
classify(sdi)
classify(ssem)
classify(hynix)

classify(greencross)
classify(daewoong)
classify(yoohan)
classify(hanmi)

,회사명,거래소코드,회계년도,자산(*),자본(*),* 수권주식수,*주당 액면가액,* 발행한 주식총수(*),보통주,우선주,지배기업주주지분(*),자본금(*),보통주자본금,우선주자본금,신종자본증권,자본잉여금(*),주식발행초과금,배당건설이자,주식매입선택권,감자차익,보험차익,합병차익,자산수증이익,채무면제이익,자기주식처분이익,전환권대가,신주인수권대가,교환권대가,전환권재매입이익,기타자본잉여금,재평가적립금(2001년 이전),기타자본(*),자기주식,자기주식처분손실,주식할인발행차금,주식매입선택권.1,감자차손,파생상품평가이익,파생상품평가손실,전환권재매입손실,기타,기타포괄손익누계액(*),금융자산평가손익,매도가능금융자산평가손익,관계기업 등 기타포괄손익,환산 관련 외환차이 적립금(환율변동차이),해외사업장순투자위험회피 적립금,확정급여제도의 보험수리적 손익 적립금,확정급여제도의 재측정요소,현금흐름위험회피 적립금,주식기준보상 적립금,재평가잉여금,기타.1,이익잉여금(결손금)(*),* (지배)당기순손익,이익준비금,재무구조개선적립금,기업합리화적립금,기타법정적립금,재평가손익,보험수리적손익,기타이익잉여금,미처분이익잉여금(결손금),*전기이월미처분이익잉여금(결손금),비지배주주지분,* (비지배)당기순손익,*비지배주주지분초과손실액,부채(*),* (정상)영업손익(보고서기재)(3개월),* (정상)영업손익(계산수치)(3개월),기타이익(*)(3개월),이자수익(3개월),배당금수익(3개월),외환거래이익(*)(3개월),외환차익(3개월),외환환산이익(3개월),기타외화거래이익(3개월),보험차익(3개월),퇴직연금운용이익(3개월),전기오류수정이익(3개월),기타(3개월),기타손실(*)(3개월),이자비용(3개월),외환거래손실(*)(3개월),외환차손(3개월),외화환산손실(3개월),기타외환거래손실(3개월),법인세추납액(3개월),퇴직연금운용손실(3개월),전기오류수정손실(3개월),기타(3개월).1,법인세비용차감전순이익(손실)(3개월),법인세비용(3개월),종속회사 매수일전 순손익(3개월),처분된 종속회사 순손익(3개월),계속영업이익(손실)(3개월),중단영업이익(손실)(3개월),*중단영업손익 법인세효과(3개월),당기순이익(손실)(3개월),기타포괄손익(*)(3개월),금융자산평가손익(3개월),매도가능금융자산평가손익(3개월),관계기업 등 기타포괄손익(3개월),환산 관련 외환차이(환율변동차이)(3개월),해외사업장순투자위험회피(3개월),현금흐름위험회피 적립금(3개월),확정급여제도의 보험수리적손익(3개월),확정급여제도의 재측정요소(3개월),주식기준보상 적립금(3개월),재평가손익(3개월),기타(3개월).2,기타포괄손익관련 법인세(3개월),총포괄손익(3개월),*(당기순손익 귀속)지배기업주주지분(3개월),*계속영업손익(3개월),*중단영업손익(3개월),*(당기순손익 귀속)비지배주주지분(3개월),*계속영업손익(3개월).1,*중단영업손익(3개월).1,*(총포괄손익 귀속)지배기업주주지분(3개월),*(총포괄손익 귀속)비지배주주지분(3개월),*주당계속영업이익(3개월),*주당순손익(3개월),*희석주당계속영업이익(3개월),*희석주당순손익(3개월),*(지배기업주주지분)주당계속영업이익(3개월),*(지배기업주주지분)주당순손익(3개월),*(지배기업주주지분)희석주당계속영업이익(3개월),*(지배기업주주지분)희석주당순손익(3개월),* 성격별 비용계정(*)(3개월),제품과 재공품의 변동(3개월),기업이 수행한 용역으로서 자본화되어 있는 부분(3개월),원재료와 저장품(소모품)의 사용액(3개월),상품의 판매(3개월),기타원가(3개월),종업원 급여비용(3개월),감가상각비와 기타상각비 및 손상차손(3개월),세금과공과(3개월),대손상각비(3개월),물류비(운송보관)(3개월),광고 및 판매촉진비(3개월),임차료 및 리스료(3개월),경상연구개발비(3개월),기타비용(3개월),기말자본금(*),기초자본금,유상증자(감자),무상증자(감자),주식매수선택권,전환사채,신주인수권부사채,주식배당,합병분할(영업양수도 등)로 인한 변동,회계정책변경누적효과,오류수정 등,연결범위의 변동,외화환산차이,기타.2,기말신종자본증권(*),기초신종자본증권,신종자본증권의 발행,기타.3,기말자본잉여금(*),기초자본잉여금,유상증자(감자).1,무상증자(감자).1,주식발행초과금.1,주식매수선택권.1,감자차손익,보험차익.1,자기주식처분손익,전환권대가(전환사채),신주인수권대가(신주인수권부사채),교환권대가(교환사채),전환권재매입손익,결손금처리,지분법자본잉여금,합병분할(영업양수도 등)로 인한 변동.1,회계정책변경누적효과.1,오류수정 등.1,연결범위의 변동.1,외화환산차이.1,기타.4,기말기타자본(*),기초기타자본,자기주식.1,자기주식처분손실.1,주식할인발행차금.1,주식매입선택권.2,파생상품평가이익.1,파생상품평가손실.1,출자전환채무,신주청약증거금,감자차손익.1,전환권재매입손익.1,지분법자본조정,합병분할(영업양수도 등)로 인한 변동.2,회계정책변경누적효과.2,오류수정 등.2,연결범위의 변동.2,외화환산차이.2,기타.5,기말기타포괄손익누계액(*),기초기타포괄손익누계액,금융자산평가손익.1,매도가능금융자산 평가손익,관계기업 등 기타포괄손익.1,환산 관련 외환차이 적립금(환율변동차이).1,해외사업장순투자위험회피 적립금.1,확정급여제도의 보험수리적 손익 적립금.1,확정급여제도의 재측정요소.1,현금흐름위험회피 적립금.1,주식기준보상 적립금.1,재평가잉여금.1,합병분할(영업양수도 등)로 인한 변동.3,회계정책변경누적효과.3,오류수정 등.3,연결범위의 변동.3,외화환산차이.3,기타.6,기말이익잉여금(결손금)(*),처분후 이익잉여금(결손금)(*),수정후 이익잉여금(결손금)(*),기초이익잉여금(결손금),회계정책변경누적효과.4,전기오류수정손익,기타.7,(연차배당),(신종자본증권 분배금),주식할인발행차금의 상각,자기주식처분손실의 상각,결손금처리.1,기타이익잉여금처분액,(중간배당),확정급여제도의 재측정요소.2,기타포괄손익,당기순이익(총포괄손익),보험수리적손익.1,재평가손익.1,지분법이익잉여금,합병분할(영업양수도 등)로 인한 변동.4,연결범위의 변동.4,외화환산차이.4,기타.8,기말비지배주주지분(*),처분후 비지배주주지분(*),수정후 비지배주주지분(*),기초 비지배주주지분,회계정책변경누적효과.5,전기오류수정손익.1,기타.9,(종속회사의 배당금),기타.10,(중간배당).1,당기순이익(총포괄손익).1,재평가잉여금.2,금융자산평가손익.2,매도가능금융자산 평가손익.1,관계기업 등 지분법평가손익,환산 관련 외환차이 적립금(환율변동차이).2,해외사업장순투자위험회피 적립금.2,확정급여제도의 보험수리적 손익 적립금.2,확정급여제도의 재측정요소.3,현금흐름위험회피 적립금.2,주식기준보상 적립금.2,기타포괄손익누계액,합병분할로 인한 변동,연결범위의 변동.5,외화환산차이.5,기타.11,합계,영업활동으로 인한 현금흐름(간접법)(*),*(직접법)총현금유입(*),재화의 판매와 용역제공에 따른 현금유입,"수입보험료, 보험금, 연금 및 기타 급부금관련 현금유입",이자수취액,배당금수취액,법인세환급,기타현금유입액,*(직접법)총현금유출(*),종업원과 관련하여 직·간접으로 발생하는 현금유출,"수입보험료, 보험금, 연금 및 기타 급부금관련 현금유출",이자비용유출액,배당금지급액,법인세납부,단기매매목적으로 보유하는 계약에서 발생하는 현금유출,기타현금유출액,당기순손익,법인세비용차감전순손익,현금 유출이 없는 비용 등 가산(*),전기오류수정손익.2,비지배주주지분순손익,보험수리적손실,기타.12,(현금 유입이 없는 수익 등 차감)(*),전기오류수정손익.3,비지배주주지분순손익.1,보험수리적이익,기타.13,영업활동으로 인한 자산부채 변동(운전자본 변동)(*),자산의 감소(증가)(*),부채의 증가(감소)(*),정부보조금 등의 변동,기타 운전자본의 변동,*영업에서 창출된 현금흐름,이자수입,이자지급(-),배당금수입,배당금지급(-),법인세환입,법인세납부(-),중단영업관련 현금흐름,투자활동으로 인한 현금흐름(*),투자활동으로 인한 현금유입액(*),정부보조금 등의 변동.1,종속기업 및 기타사업의 지배력 관련한 현금흐름,합병분할(양수도등)으로 인한 현금유입액,기타투자활동으로 인한 현금유입액,(투자활동으로 인한 현금유출액)(*),정부보조금 등의 변동.2,종속기업 및 기타사업의 지배력 관련한 현금흐름.1,

In [13]:
lg

,회사명,거래소코드,회계년도,자산(*),자본(*),* 수권주식수,*주당 액면가액,* 발행한 주식총수(*),보통주,우선주,지배기업주주지분(*),자본금(*),보통주자본금,우선주자본금,신종자본증권,자본잉여금(*),주식발행초과금,배당건설이자,주식매입선택권,감자차익,보험차익,합병차익,자산수증이익,채무면제이익,자기주식처분이익,전환권대가,신주인수권대가,교환권대가,전환권재매입이익,기타자본잉여금,재평가적립금(2001년 이전),기타자본(*),자기주식,자기주식처분손실,주식할인발행차금,감자차손,파생상품평가이익,파생상품평가손실,전환권재매입손실,기타,기타포괄손익누계액(*),금융자산평가손익,매도가능금융자산평가손익,관계기업 등 기타포괄손익,환산 관련 외환차이 적립금(환율변동차이),해외사업장순투자위험회피 적립금,확정급여제도의 보험수리적 손익 적립금,확정급여제도의 재측정요소,현금흐름위험회피 적립금,주식기준보상 적립금,재평가잉여금,기타.1,이익잉여금(결손금)(*),* (지배)당기순손익,이익준비금,재무구조개선적립금,기업합리화적립금,기타법정적립금,재평가손익,보험수리적손익,기타이익잉여금,미처분이익잉여금(결손금),*전기이월미처분이익잉여금(결손금),비지배주주지분,* (비지배)당기순손익,*비지배주주지분초과손실액,부채(*),* (정상)영업손익(보고서기재),* (정상)영업손익(계산수치),기타이익(*),이자수익,배당금수익,외환거래이익(*),외환차익,외환환산이익,기타외화거래이익,보험차익.1,퇴직연금운용이익,전기오류수정이익,기타.2,기타손실(*),이자비용,외환거래손실(*),외환차손,외화환산손실,기타외환거래손실,법인세추납액,퇴직연금운용손실,전기오류수정손실,기타.3,법인세비용차감전순이익(손실),법인세비용,종속회사 매수일전 순손익,처분된 종속회사 순손익,계속영업이익(손실),중단영업이익(손실),*중단영업손익 법인세효과,당기순이익(손실),기타포괄손익(*),금융자산평가손익.1,매도가능금융자산평가손익.1,관계기업 등 기타포괄손익.1,환산 관련 외환차이(환율변동차이),해외사업장순투자위험회피,현금흐름위험회피 적립금.1,확정급여제도의 보험수리적손익,확정급여제도의 재측정요소.1,주식기준보상 적립금.1,재평가손익.1,기타.4,기타포괄손익관련 법인세,총포괄손익,*(당기순손익 귀속)지배기업주주지분,*계속영업손익,*중단영업손익,*(당기순손익 귀속)비지배주주지분,*계속영업손익.1,*중단영업손익.1,*(총포괄손익 귀속)지배기업주주지분,*(총포괄손익 귀속)비지배주주지분,*주당계속영업이익,*주당순손익,*희석주당계속영업이익,*희석주당순손익,*(지배기업주주지분)주당계속영업이익,*(지배기업주주지분)주당순손익,*(지배기업주주지분)희석주당계속영업이익,*(지배기업주주지분)희석주당순손익,* 성격별 비용계정(*),제품과 재공품의 변동,기업이 수행한 용역으로서 자본화되어 있는 부분,원재료와 저장품(소모품)의 사용액,상품의 판매,기타원가,종업원 급여비용,감가상각비와 기타상각비 및 손상차손,세금과공과,대손상각비,물류비(운송보관),광고 및 판매촉진비,임차료 및 리스료,경상연구개발비,기타비용,기말자본금(*),기초자본금,유상증자(감자),무상증자(감자),주식매수선택권,전환사채,신주인수권부사채,주식배당,합병분할(영업양수도 등)로 인한 변동,회계정책변경누적효과,오류수정 등,연결범위의 변동,외화환산차이,기타.5,기말신종자본증권(*),기초신종자본증권,신종자본증권의 발행,기타.6,기말자본잉여금(*),기초자본잉여금,유상증자(감자).1,무상증자(감자).1,주식발행초과금.1,주식매수선택권.1,감자차손익,보험차익.2,자기주식처분손익,전환권대가(전환사채),신주인수권대가(신주인수권부사채),교환권대가(교환사채),전환권재매입손익,결손금처리,지분법자본잉여금,합병분할(영업양수도 등)로 인한 변동.1,회계정책변경누적효과.1,오류수정 등.1,연결범위의 변동.1,외화환산차이.1,기타.7,기말기타자본(*),기초기타자본,자기주식.1,자기주식처분손실.1,주식할인발행차금.1,주식매입선택권.2,파생상품평가이익.1,파생상품평가손실.1,출자전환채무,신주청약증거금,감자차손익.1,전환권재매입손익.1,지분법자본조정,합병분할(영업양수도 등)로 인한 변동.2,회계정책변경누적효과.2,오류수정 등.2,연결범위의 변동.2,외화환산차이.2,기타.8,기말기타포괄손익누계액(*),기초기타포괄손익누계액,금융자산평가손익.2,매도가능금융자산 평가손익,관계기업 등 기타포괄손익.2,환산 관련 외환차이 적립금(환율변동차이).1,해외사업장순투자위험회피 적립금.1,확정급여제도의 보험수리적 손익 적립금.1,확정급여제도의 재측정요소.2,현금흐름위험회피 적립금.2,주식기준보상 적립금.2,재평가잉여금.1,합병분할(영업양수도 등)로 인한 변동.3,회계정책변경누적효과.3,오류수정 등.3,연결범위의 변동.3,외화환산차이.3,기타.9,기말이익잉여금(결손금)(*),처분후 이익잉여금(결손금)(*),수정후 이익잉여금(결손금)(*),기초이익잉여금(결손금),회계정책변경누적효과.4,전기오류수정손익,기타.10,(연차배당),(신종자본증권 분배금),주식할인발행차금의 상각,자기주식처분손실의 상각,결손금처리.1,기타이익잉여금처분액,(중간배당),확정급여제도의 재측정요소.3,기타포괄손익,당기순이익(총포괄손익),보험수리적손익.1,재평가손익.2,지분법이익잉여금,합병분할(영업양수도 등)로 인한 변동.4,연결범위의 변동.4,외화환산차이.4,기타.11,기말비지배주주지분(*),처분후 비지배주주지분(*),수정후 비지배주주지분(*),기초 비지배주주지분,회계정책변경누적효과.5,전기오류수정손익.1,기타.12,(종속회사의 배당금),기타.13,(중간배당).1,당기순이익(총포괄손익).1,재평가잉여금.2,금융자산평가손익.3,매도가능금융자산 평가손익.1,관계기업 등 지분법평가손익,환산 관련 외환차이 적립금(환율변동차이).2,해외사업장순투자위험회피 적립금.2,확정급여제도의 보험수리적 손익 적립금.2,확정급여제도의 재측정요소.4,현금흐름위험회피 적립금.3,주식기준보상 적립금.3,기타포괄손익누계액,합병분할로 인한 변동,연결범위의 변동.5,외화환산차이.5,기타.14,합계,영업활동으로 인한 현금흐름(간접법)(*),*(직접법)총현금유입(*),재화의 판매와 용역제공에 따른 현금유입,"수입보험료, 보험금, 연금 및 기타 급부금관련 현금유입",이자수취액,배당금수취액,법인세환급,기타현금유입액,*(직접법)총현금유출(*),종업원과 관련하여 직·간접으로 발생하는 현금유출,"수입보험료, 보험금, 연금 및 기타 급부금관련 현금유출",이자비용유출액,배당금지급액,법인세납부,단기매매목적으로 보유하는 계약에서 발생하는 현금유출,기타현금유출액,당기순손익,법인세비용차감전순손익,현금 유출이 없는 비용 등 가산(*),전기오류수정손익.2,비지배주주지분순손익,보험수리적손실,기타.15,(현금 유입이 없는 수익 등 차감)(*),전기오류수정손익.3,비지배주주지분순손익.1,보험수리적이익,기타.16,영업활동으로 인한 자산부채 변동(운전자본 변동)(*),자산의 감소(증가)(*),부채의 증가(감소)(*),정부보조금 등의 변동,기타 운전자본의 변동,*영업에서 창출된 현금흐름,이자수입,이자지급(-),배당금수입,배당금지급(-),법인세환입,법인세납부(-),중단영업관련 현금흐름,투자활동으로 인한 현금흐름(*),투자활동으로 인한 현금유입액(*),정부보조금 등의 변동.1,종속기업 및 기타사업의 지배력 관련한 현금흐름,합병분할(양수도등)으로 인한 현금유입액,기타투자활동으로 인한 현금유입액,(투자활동으로 인한 현금유출액)(*),정부보조금 등의 변동.2,종속기업 및 기타사업의 지배력 관련한 현금흐름.1,합병분할(양수도등)으로 인한 현금유출액,기타투자활동으로 인한 현금유출액,이자수입.1,이자지급(-).1,배당금수입.1,배당금지급(-).1,법인세환입.1,법인세납부(-).1,중단영업관련 현금흐름.1,재무활동으로 인한 현금흐름(*),재무활동으로 인한 현금유입액(*),기타부채의 증가,유상증자,자본금의 증가,자본잉여금의 증가,종속회사의 증자 등,이익잉여금의 증가,자기주식의 처분,주식매입선택권의 행사,연결자본거래로인한현금유입액,종속기업 소유지분 변동에 따른 현금흐름,비지배주주지분의 증가,정부보조금 등의 변동.3,환율변동으로인한 차이조정,합병분할(양수도등)으로 인한 현금유입액.1,기타재무활동으로 인한 현금유입액,(재무활동으로 인한 현금유출액)(*),유상감자,자본금의

In [14]:
lg = lg.iloc[:-1,:]
sdi = sdi.iloc[:-1,:]
ssem = ssem.iloc[:-1,:]
hynix = hynix.iloc[:-1,:]

greencross = greencross.iloc[:-1,:]
daewoong = daewoong.iloc[:-1,:]
yoohan = yoohan.iloc[:-1,:]
hanmi = hanmi.iloc[:-1,:]

# 산업별 변수 <3개월 후 종가>와의 상관관계 분석

In [15]:
# 밑에 함수로 해보기!!!!

In [16]:
lg_corr = lg.corr(method = 'pearson')
df_lg = pd.DataFrame(lg_corr['3개월후종가']).reset_index().sort_values('3개월후종가', ascending = False)
df_lg.columns = ['지표','3개월후종가']

sdi_corr = sdi.corr(method = 'pearson')
df_sdi = pd.DataFrame(sdi_corr['3개월후종가']).reset_index().sort_values('3개월후종가', ascending = False)
df_sdi.columns = ['지표','3개월후종가']

ssem_corr = ssem.corr(method = 'pearson')
df_ssem = pd.DataFrame(ssem_corr['3개월후종가']).reset_index().sort_values('3개월후종가', ascending = False)
df_ssem.columns = ['지표','3개월후종가']

hynix_corr = hynix.corr(method = 'pearson')
df_hynix = pd.DataFrame(hynix_corr['3개월후종가']).reset_index().sort_values('3개월후종가', ascending = False)
df_hynix.columns = ['지표','3개월후종가']

# 산업별 상관관계 데이터프레임

In [17]:
pd.concat([df_lg.reset_index(drop = True), df_sdi.reset_index(drop = True), df_ssem.reset_index(drop = True), df_hynix.reset_index(drop = True)],axis = 1)

,지표,3개월후종가,지표,3개월후종가,지표,3개월후종가,지표,3개월후종가
0,3개월후종가,1.000000,3개월후종가,1.000000,3개월후종가,1.000000,3개월후종가,1.000000
1,최고가,0.767493,최고가,0.820795,최고가,0.827799,종가,0.948256
2,종가,0.747660,종가,0.809146,종가,0.818502,최고가,0.947936
3,최저가,0.712841,최저가,0.781223,최저가,0.801165,최저가,0.944148
4,거래대금,0.686572,회계정책변경누적효과.4,0.593481,거래대금,0.730220,* (정상)영업손익(계산수치),0.930893
5,거래량,0.598758,정부보조금 등의 변동.1,0.580991,해외사업장순투자위험회피 적립금,0.703484,* (정상)영업손익(보고서기재),0.928501
6,기말이익잉여금(결손금)(*),0.539389,미처분이익잉여금(결손금),0.474870,* (정상)영업손익(보고서기재),0.659242,법인세비용차감전순이익(손실),0.918030
7,이익잉여금(결손금)(*),0.539389,매도가능금융자산 평가손익.1,0.461993,* (정상)영업손익(계산수치),0.655114,*(지배기업주주지분)주당순손익,0.915234
8,확정급여제도의 재측정요소.3,0.533428,이자지급(-),0.453903,거래량,0.593055,*(지배기업주주지분)주당계속영업이익,0.915234
9,*(당기순손익 귀속)지배기업주주지분,0.517290,* (정상)영업손익(계산수치),0.441103,기초자본잉여금,0.545441,*(당기순손익 귀속)지배기업주주지분,0.914956


In [18]:
list_corr = []
order = 15 # 기업별로 상위 15개 상관관계 변수 추출

for i in range(len(df_lg.index[:order])):
    list_corr.append(df_lg.index[:order][i])
    list_corr.append(df_sdi.index[:order][i])
    list_corr.append(df_ssem.index[:order][i])
    list_corr.append(df_hynix.index[:order][i])

variable = pd.Series(list_corr).value_counts()
variable = pd.DataFrame(variable).reset_index()
variable.columns = ['var_index', 'number']

In [19]:
var = []
for i in range(len(variable)):
    var.append(df_lg[df_lg.index == variable['var_index'][i]]['지표'].values[0])
var = pd.DataFrame(var)
var.columns = ['var']

In [20]:
corr = pd.concat([variable,var], axis = 1)
corr = corr[['var_index','var','number']]
corr.head(20)

,var_index,var,number
0,376,3개월후종가,4
1,373,종가,4
2,372,최저가,4
3,371,최고가,4
4,66,* (정상)영업손익(계산수치),3
5,375,거래대금,3
6,65,* (정상)영업손익(보고서기재),2
7,110,*(당기순손익 귀속)지배기업주주지분,2
8,116,*(총포괄손익 귀속)지배기업주주지분,2
9,88,법인세비용차감전순이익(손실),2


# 위 자료를 보고 변수 선정

In [21]:
deep_lg = lg[corr['var'][4:19]]
deep_lg['수익률상승하락'] = lg['수익률상승하락']
deep_lg

,* (정상)영업손익(계산수치),거래대금,* (정상)영업손익(보고서기재),*(당기순손익 귀속)지배기업주주지분,*(총포괄손익 귀속)지배기업주주지분,법인세비용차감전순이익(손실),*(지배기업주주지분)주당순손익,총포괄손익,당기순이익(손실),거래량,미처분이익잉여금(결손금),기말자본잉여금(*),회계정책변경누적효과.4,자본잉여금(*),정부보조금 등의 변동.1,수익률상승하락
0,41554974,644359793314,42395332,47423280,41924583,41080355,2765,42143168,47401856,5785616,265369000,494216917,0,494216917,1475379,1.0
1,79851762,1009752795890,86912500,101219879,113231727,62848450,5689,112908596,101442004,6159012,366534000,840990251,0,840990251,9768001,0.0
2,68547655,1165922704000,63152758,66598571,54534243,59874975,3311,54713459,66638920,8400827,427961000,830529899,0,830529899,4122189,0.0
3,-36793272,549851086500,-36004758,-19650321,-23373899,-55127153,-1286,-23375019,-19651440,4155810,0,845623497,0,845623497,4606499,0.0
4,-8786085,597488403500,-7369064,-9529000,-14478270,-26674417,-473,-14479538,-9530267,5105469,385654768,847337998,0,847337998,0,0.0
5,9623556,199759286700,5914429,-6974465,-9701523,-15194170,-346,-9702461,-6975403,2080293,378690000,848407283,0,848407283,0,0.0
6,-15828492,208123631400,-5366696,-35551689,-13256556,-43309363,-1764,-13257642,-35552775,3298925,343096000,848405333,0,848405333,0,1.0
7,-27898047,193145613600,-60014817,-93302093,-103580778,-89294631,-4628,-103594761,-93316075,2724149,241422000,848398733,0,848398733,0,1.0
8,18258822,478587655600,23863085,-2431835,-5343403,43058,-121,-5343820,-2432253,4689082,239037376,849197759,0,849197759,0,0.0
9,15243876,183233907400,19046931,4459358,6102180,-7965601,221,6102098,4459276,2138257,243492000,849192486,0,849192486,0,1.0


In [22]:
deep_sdi = sdi[corr['var'][4:19]]
deep_sdi['수익률상승하락'] = sdi['수익률상승하락']
deep_sdi

,* (정상)영업손익(계산수치),거래대금,* (정상)영업손익(보고서기재),*(당기순손익 귀속)지배기업주주지분,*(총포괄손익 귀속)지배기업주주지분,법인세비용차감전순이익(손실),*(지배기업주주지분)주당순손익,총포괄손익,당기순이익(손실),거래량,미처분이익잉여금(결손금),기말자본잉여금(*),회계정책변경누적효과.4,자본잉여금(*),정부보조금 등의 변동.1,수익률상승하락
0,35045876,1117298042500,64656618,46696585,85251865,63912711,1087,90474300,56006983,8108564,177410565,1280969475,0,1280969475,0,1.0
1,48192963,2495911626000,83647666,68422306,229152676,98805464,1588,251112821,78221511,14286450,245832871,1291612414,0,1291612414,0,0.0
2,121854634,1843565266625,123809638,159659249,360269977,182507486,3697,364994020,167510190,11296132,415666024,1299819561,0,1299819561,0,1.0
3,29130355,1779554946000,14698275,81324629,395606111,77374652,1824,399720684,83373658,10389944,544059037,1263797851,0,1263797851,0,0.0
4,8595066,2146377902500,60347343,78712662,25690710,111688509,1820,27120354,83464922,12924052,324009540,1256319704,0,1256319704,0,1.0
5,55546053,1898624931500,89228461,86512997,194287950,130549033,1998,194849971,94821566,11360780,410522537,1256723483,0,1256723483,0,0.0
6,27992220,1461671086500,43010471,99956639,-44879453,135887011,2308,-18312288,111570831,12380358,510479176,1256741730,0,1256741730,0,1.0
7,17834633,1151550699000,11127476,54926932,-50482302,99974960,1214,-46708398,61197597,8321983,536711370,1258119974,0,1258119974,0,1.0
8,16869079,988248477000,67037476,112010791,240696110,165202801,2584,246599051,118809346,7046004,721988414,1258392571,0,1258392571,0,1.0
9,82628588,657899482000,83970453,105329378,-75868792,166302103,2429,-65347267,116147767,4305587,827317792,1258440118,0,1258440118,0,0.0


In [23]:
deep_ssem = ssem[corr['var'][4:19]]
deep_ssem['수익률상승하락'] = ssem['수익률상승하락']
deep_ssem

,* (정상)영업손익(계산수치),거래대금,* (정상)영업손익(보고서기재),*(당기순손익 귀속)지배기업주주지분,*(총포괄손익 귀속)지배기업주주지분,법인세비용차감전순이익(손실),*(지배기업주주지분)주당순손익,총포괄손익,당기순이익(손실),거래량,미처분이익잉여금(결손금),기말자본잉여금(*),회계정책변경누적효과.4,자본잉여금(*),정부보조금 등의 변동.1,수익률상승하락
0,110485025,1905902108600,119107203,123406193,38061062,196112040,1597,64989869,152489335,18089393,314383217,1048380070,0,1048380070,0,1.0
1,280376487,2327381381400,311188142,180071656,389601646,302836818,2328,449785657,233078412,15957107,494454873,1047696983,0,1047696983,0,0.0
2,273757102,3229847980500,263995716,159066396,259135952,246980151,2056,295404088,198789893,27429377,628849119,1047104999,0,1047104999,0,1.0
3,70031503,2103988272000,83822473,92109781,253413716,84767776,1189,245419974,82358099,16677355,720656894,1046242505,0,1046242505,0,0.0
4,81070289,3235184168500,92114453,85567748,41510027,90121153,1105,53467521,99197338,27108459,0,1045223176,0,1045223176,0,0.0
5,71856178,1207055780650,87096950,54124577,51768809,73487344,698,49761639,55715777,13038907,0,1045177182,0,1045177182,0,0.0
6,43499277,1342787249900,68910715,64925061,-43226351,51406318,838,-28072247,67072033,19922132,0,1045723483,0,1045723483,0,1.0
7,81389125,1582787670700,72735086,144855760,58125300,149021846,1870,79943487,170396591,18663868,496747434,1045746729,0,1045746729,0,1.0
8,94789768,1194645946700,106665186,69506236,123840530,98812512,897,123889794,70427941,12176773,0,1045015595,0,1045015595,0,1.0
9,156662079,1110972308500,156172550,135164513,90606795,144688102,1743,90286994,135324658,10680064,0,1045022349,0,1045022349,0,0.0


In [24]:
deep_hynix = hynix[corr['var'][4:19]]
deep_hynix['수익률상승하락'] = hynix['수익률상승하락']
deep_hynix

,* (정상)영업손익(계산수치),거래대금,* (정상)영업손익(보고서기재),*(당기순손익 귀속)지배기업주주지분,*(총포괄손익 귀속)지배기업주주지분,법인세비용차감전순이익(손실),*(지배기업주주지분)주당순손익,총포괄손익,당기순이익(손실),거래량,미처분이익잉여금(결손금),기말자본잉여금(*),회계정책변경누적효과.4,자본잉여금(*),정부보조금 등의 변동.1,수익률상승하락
0,332337000,6478479539100,322829000,273264000,202640000,281418000,463,201528000,273540000,221844506,3768834000,1203046000,0,1203046000,0,0.0
1,257148000,6354361623460,446889000,471087000,423581000,472475000,796,423504000,472996000,236066809,4239908000,1230979000,0,1230979000,0,0.0
2,-291826000,5679510775440,-276759000,-561100000,-291392000,-551519000,0,-290212000,-562620000,279111657,3678800000,1230979000,0,1230979000,0,1.0
3,-192441000,2826941308190,-167485000,-239893000,-321882000,-157226000,-96,-321941000,-239888000,127788137,3310963000,1229052000,0,1229052000,0,1.0
4,-267436000,3164699841500,-259981000,-271119000,-302868000,-271028000,-420,-303022000,-271223000,106528104,0,3045857000,0,3045857000,0,0.0
5,1146000,2904812064880,22804000,-53158000,-40759000,-43875000,-77,-40867000,-53307000,125129001,0,3049248000,0,3049248000,0,0.0
6,-31193000,2580989738910,-15073000,1885000,-73046000,5184000,3,-72979000,2041000,114154941,0,3049173000,0,3049173000,153000,1.0
7,70135000,2069281014710,24901000,163506000,-47594000,110418000,243,-47528000,163694000,80286983,3068905000,3053874000,0,3053874000,153000,1.0
8,316951000,2573447265700,316951000,178595000,303623000,208217000,257,303855000,178713000,90828794,0,3054075000,0,3054075000,0,1.0
9,1113603000,2879298818820,1113603000,946807000,1099329000,1012802000,1363,1099386000,946761000,92187155,0,3066076000,0,3066076000,0,0.0


In [25]:
deep_IT = pd.concat([deep_lg,deep_hynix,deep_ssem,deep_sdi], ignore_index=True)
deep_IT

,* (정상)영업손익(계산수치),거래대금,* (정상)영업손익(보고서기재),*(당기순손익 귀속)지배기업주주지분,*(총포괄손익 귀속)지배기업주주지분,법인세비용차감전순이익(손실),*(지배기업주주지분)주당순손익,총포괄손익,당기순이익(손실),거래량,미처분이익잉여금(결손금),기말자본잉여금(*),회계정책변경누적효과.4,자본잉여금(*),정부보조금 등의 변동.1,수익률상승하락
0,41554974,644359793314,42395332,47423280,41924583,41080355,2765,42143168,47401856,5785616,265369000,494216917,0,494216917,1475379,1.0
1,79851762,1009752795890,86912500,101219879,113231727,62848450,5689,112908596,101442004,6159012,366534000,840990251,0,840990251,9768001,0.0
2,68547655,1165922704000,63152758,66598571,54534243,59874975,3311,54713459,66638920,8400827,427961000,830529899,0,830529899,4122189,0.0
3,-36793272,549851086500,-36004758,-19650321,-23373899,-55127153,-1286,-23375019,-19651440,4155810,0,845623497,0,845623497,4606499,0.0
4,-8786085,597488403500,-7369064,-9529000,-14478270,-26674417,-473,-14479538,-9530267,5105469,385654768,847337998,0,847337998,0,0.0
5,9623556,199759286700,5914429,-6974465,-9701523,-15194170,-346,-9702461,-6975403,2080293,378690000,848407283,0,848407283,0,0.0
6,-15828492,208123631400,-5366696,-35551689,-13256556,-43309363,-1764,-13257642,-35552775,3298925,343096000,848405333,0,848405333,0,1.0
7,-27898047,193145613600,-60014817,-93302093,-103580778,-89294631,-4628,-103594761,-93316075,2724149,241422000,848398733,0,848398733,0,1.0
8,18258822,478587655600,23863085,-2431835,-5343403,43058,-121,-5343820,-2432253,4689082,239037376,849197759,0,849197759,0,0.0
9,15243876,183233907400,19046931,4459358,6102180,-7965601,221,6102098,4459276,2138257,243492000,849192486,0,849192486,0,1.0


In [26]:
# 표준화

from sklearn.preprocessing import StandardScaler

scaler_lg = StandardScaler()
scale_lg = pd.DataFrame(scaler_lg.fit_transform(deep_lg.iloc[:,:-1]))


# IT 업종 표준화

scaler_IT = StandardScaler()
scale_IT = pd.DataFrame(scaler_IT.fit_transform(deep_IT.iloc[:,:-1]))


# 원래 값으로 변환
# scaler_lg.inverse_transform(scale_lg)

In [27]:
scale_IT

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,-0.331871,-0.587695,-0.327937,-0.343896,-0.340080,-0.353957,0.453414,-0.342465,-0.347509,-0.496585,-0.380703,-1.033902,-0.119816,-1.033902,-0.061426
1,-0.289254,-0.353960,-0.279383,-0.268408,-0.241606,-0.330094,1.342690,-0.244767,-0.271647,-0.488470,-0.364338,-0.809010,-0.119816,-0.809010,0.967469
2,-0.301833,-0.254061,-0.305297,-0.316989,-0.322666,-0.333354,0.619469,-0.325111,-0.320504,-0.439749,-0.354401,-0.815794,-0.119816,-0.815794,0.266973
3,-0.419055,-0.648150,-0.413447,-0.438014,-0.430256,-0.459422,-0.778617,-0.432919,-0.441639,-0.532006,-0.423632,-0.806005,-0.119816,-0.806005,0.327063
4,-0.387889,-0.617677,-0.382215,-0.423811,-0.417971,-0.428231,-0.531359,-0.420638,-0.427431,-0.511367,-0.361245,-0.804893,-0.119816,-0.804893,-0.244481
5,-0.367403,-0.872097,-0.367726,-0.420227,-0.411374,-0.415646,-0.492734,-0.414043,-0.423844,-0.577114,-0.362371,-0.804200,-0.119816,-0.804200,-0.244481
6,-0.395726,-0.866746,-0.380031,-0.460326,-0.416284,-0.446467,-0.923991,-0.418951,-0.463961,-0.550629,-0.368129,-0.804201,-0.119816,-0.804201,-0.244481
7,-0.409157,-0.876327,-0.439635,-0.541362,-0.541020,-0.496877,-1.795019,-0.543669,-0.545050,-0.563121,-0.384577,-0.804205,-0.119816,-0.804205,-0.244481
8,-0.357794,-0.693736,-0.348150,-0.413853,-0.405356,-0.398943,-0.424305,-0.408025,-0.417466,-0.520417,-0.384963,-0.803687,-0.119816,-0.803687,-0.244481
9,-0.361149,-0.882668,-0.353403,-0.404183,-0.389550,-0.407722,-0.320293,-0.392223,-0.407792,-0.575854,-0.384242,-0.803690,-0.119816,-0.803690,-0.244481


# 샤비어함수 및 히든레이어 5개

# 및 Validation

In [100]:
# Review : Learning rate and Evaluation
import tensorflow as tf
import random
import matplotlib.pyplot as plt

tf.set_random_seed(777)  # reproducibility
tf.reset_default_graph()


x_data = scale_IT.iloc[:,:-1]
y_data = deep_IT.iloc[:,[-1]]


# parameters
placeholder_num = len(x_data.columns)
learning_rate = 0.001




X = tf.placeholder(tf.float32, [None, placeholder_num])
Y = tf.placeholder(tf.float32, [None, 1])
keep_prob = tf.placeholder(tf.float32)


W1 = tf.get_variable("W1", shape=[placeholder_num, 32], initializer=tf.contrib.layers.xavier_initializer())
b1 = tf.Variable(tf.random_normal([32]))
L1 = tf.nn.relu(tf.matmul(X, W1) + b1)
L1 = tf.nn.dropout(L1, keep_prob=keep_prob)


W2 = tf.get_variable("W2", shape=[32, 64], initializer=tf.contrib.layers.xavier_initializer())
b2 = tf.Variable(tf.random_normal([64]))
L2 = tf.nn.relu(tf.matmul(L1, W2) + b2)
L2 = tf.nn.dropout(L2, keep_prob=keep_prob)

W3 = tf.get_variable("W3", shape=[64, 64], initializer=tf.contrib.layers.xavier_initializer())
b3 = tf.Variable(tf.random_normal([64]))
L3 = tf.nn.relu(tf.matmul(L2, W3) + b3)
L3 = tf.nn.dropout(L3, keep_prob=keep_prob)


W4 = tf.get_variable("W4", shape=[64, 32], initializer=tf.contrib.layers.xavier_initializer())
b4 = tf.Variable(tf.random_normal([32]))
L4 = tf.nn.relu(tf.matmul(L3, W4) + b4)
L4 = tf.nn.dropout(L4, keep_prob=keep_prob)


W5 = tf.get_variable("W5", shape=[32, 1], initializer=tf.contrib.layers.xavier_initializer())
b5 = tf.Variable(tf.random_normal([1]))
L5 = tf.nn.relu(tf.matmul(L4, W5) + b5)

# hypothesis = tf.matmul(L4, W5) + b5

# define cost/loss & optimizer

# Hypothesis using sigmoid: tf.div(1., 1. + tf.exp(tf.matmul(X, W)))
hypothesis = tf.sigmoid(tf.matmul(L4, W5) + b5)

# cost/loss function
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) *
                       tf.log(1 - hypothesis))

train = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
# train = tf.train.AdamOptimizer(1e-4).minimize(cost)

# Accuracy computation
# True if hypothesis>0.5 else False
predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [101]:
# Launch graph

validation_number = 20


with tf.Session() as sess:
    # Initialize TensorFlow variables
    sess.run(tf.global_variables_initializer())

    for step in tqdm_notebook(range(5001)):
        cost_val, _ = sess.run([cost, train], feed_dict={X: x_data.iloc[:-validation_number,:], Y: y_data.iloc[:-validation_number,:], keep_prob: 0.8})       
        if step % 200 == 0 or step < 10 :
            print("Step : {} \t Cost : {}".format(step, cost_val))    

    # Accuracy report
    h, c, a = sess.run([hypothesis, predicted, accuracy],
                       feed_dict={X: x_data.iloc[-validation_number:,:], Y: y_data.iloc[-validation_number:,:], keep_prob: 1.0})

Step : 0 	 Cost : 2.293527841567993
Step : 1 	 Cost : 2.196202039718628
Step : 2 	 Cost : 2.1916561126708984
Step : 3 	 Cost : 2.207504987716675
Step : 4 	 Cost : 2.071578025817871
Step : 5 	 Cost : 2.173384428024292
Step : 6 	 Cost : 2.075505495071411
Step : 7 	 Cost : 1.9462939500808716
Step : 8 	 Cost : 1.8486908674240112
Step : 9 	 Cost : 1.806688904762268
Step : 200 	 Cost : 0.8796516060829163
Step : 400 	 Cost : 0.800528883934021
Step : 600 	 Cost : 0.7490347623825073
Step : 800 	 Cost : 0.7510281205177307
Step : 1000 	 Cost : 0.7555500268936157
Step : 1200 	 Cost : 0.7070732116699219
Step : 1400 	 Cost : 0.716858983039856
Step : 1600 	 Cost : 0.7157478332519531
Step : 1800 	 Cost : 0.6985087990760803
Step : 2000 	 Cost : 0.7000705003738403
Step : 2200 	 Cost : 0.7133651375770569
Step : 2400 	 Cost : 0.7081571221351624
Step : 2600 	 Cost : 0.6702386140823364
Step : 2800 	 Cost : 0.6866579651832581
Step : 3000 	 Cost : 0.7077580094337463
Step : 3200 	 Cost : 0.728521466255188
Step

In [102]:
print('모델 정확도: ', a * 100 , '%')

모델 정확도:  55.000001192092896 %


In [103]:
# pd.DataFrame(y_data.iloc[-validation_number:,:].values, c.ravel()).reset_index()
pd.DataFrame({'real':list(y_data.iloc[-validation_number:,:].values.ravel()),'predict': list(c.ravel())})

,real,predict
0,0.0,0.0
1,0.0,0.0
2,1.0,0.0
3,0.0,0.0
4,0.0,0.0
5,1.0,0.0
6,0.0,0.0
7,0.0,0.0
8,1.0,0.0
9,0.0,0.0


In [104]:
h # hypothesis

array([[0.4659472 ],
       [0.46878868],
       [0.4652174 ],
       [0.47192046],
       [0.48744398],
       [0.48835304],
       [0.48736718],
       [0.49161443],
       [0.4851446 ],
       [0.49103844],
       [0.5031602 ],
       [0.5042136 ],
       [0.48832324],
       [0.48593378],
       [0.48627752],
       [0.48496363],
       [0.49262398],
       [0.4862261 ],
       [0.48639995],
       [0.48423678]], dtype=float32)